In [ ]:
from trace_nn import NetworkTrace, TraceObject
from model import SimpleNN
import torch
import pickle
import subprocess
import pandas as pd

# Run the main process in main.py
subprocess.run(["python", "main.py"])

# Load the network trace data saved by main.py
with open('outputs/network_trace.pkl', 'rb') as f:
    network_trace = pickle.load(f)
# Confirm trace data loaded
print(f"Trace data loaded with epochs: {list(network_trace.trace.keys())}")


### Data Preprocessing 
We want to prepare our recorded network_trace schema data by converting it into suitable formats for analysis.
First, we define a function `create_final_classification_series` which converts final classifcation results into a list for target variable y. 

We then define `convert_nn_schema_to_df` which converts our network trace schema into a multi-indexed dataframe.

In [ ]:
# We want to get the data for our y variable, which is all the final classification results 

def create_final_classification_series(network_trace):
    # Convert the final classification results dictionary to a flattened list
    final_results = network_trace.final_classification_results

    # Initialize an empty list to hold all flattened classification results
    flattened_results = []

    # Iterate over the epochs and flatten the classification results
    for epoch, result in final_results.items():
        if isinstance(result, torch.Tensor):
            # Convert the tensor to a numpy array, flatten it, and extend the flattened_results list
            flattened_results.extend(result.cpu().numpy().flatten().tolist())
        else:
            flattened_results.extend(result)

    # Convert the flattened list to a pandas Series
    series = pd.Series(flattened_results, name="Class_Result")

    return series

# Example usage
classification_series = create_final_classification_series(network_trace)
print(classification_series, 'hi')

# Example usage
classification_series = create_final_classification_series(network_trace)
display(classification_series)

In [ ]:
import pandas as pd
import torch
from helpers import get_model_level_integer

def convert_nn_schema_to_df(network_trace):
    # Create a list to hold the data for each neuron
    data = []
    # Create a list to hold the multi-level index (epoch, layer, neuron)
    index = []

    for epoch, layers in network_trace.trace.items():
        for layer_name, neurons in layers.items():
            layer_weights = network_trace.weights[epoch][layer_name]
            
            # Convert the weights tensor to a numpy array if it's a tensor
            if isinstance(layer_weights, torch.Tensor):
                layer_weights = layer_weights.numpy()
            
            num_input_connections = layer_weights.shape[1]  # This determines the number of input connections

            for neuron_id, neuron_data in neurons.items():
                neuron_id_int = get_model_level_integer(neuron_id) % layer_weights.shape[0]
                
                if neuron_id_int >= len(layer_weights):
                    print(f"Index {neuron_id_int} out of bounds for layer {layer_name} with size {len(layer_weights)}")
                    continue
                
                # Extract the weights for this specific neuron
                weight_for_neuron = layer_weights[neuron_id_int].tolist()
                
                # Pad the weights with None or a small number to ensure consistent column size
                weight_for_neuron.extend([0.000001] * (num_input_connections - len(weight_for_neuron)))
                
                # Append the weights to the data list
                data.append(weight_for_neuron)
                # Append the index (epoch, layer, neuron_id)
                index.append((epoch, layer_name, neuron_id))

    # Create the MultiIndex
    multi_index = pd.MultiIndex.from_tuples(index, names=["Epoch", "Layer", "Neuron"])

    # Create the DataFrame and return it with NaNs filled with a small number
    df = pd.DataFrame(data, index=multi_index)
    return df.fillna(0.000001)
model_df = convert_nn_schema_to_df(network_trace)
display(model_df)

### What to do about NaNs
At this point the question of what to do about missing values comes up.  For a fully connected network, the shape of the weight tensor is `(output_features, input_features)`. Unless a network is a perfect cube with all layers having the same number of output features and input features, this will result in empty values for neurons in layers that have less input features than the layer with the maximum number of input features. 

Thus, in the example of 
L_1 = {n_0, ..., n_19} with  shape: (10,20)
L_2 = {n_20, ... n_29}  with shape: (10, 10)
L_3 = {n_30} with shape: (1, 10)
... 
Layer 2 (L_2) will have 10 fewer values per neuron than its previous layer, as it will have 10 fewer input connections. 

This makes sense intuitively because a layer with 20 neurons will take 20 input connections but give 10 connections to every neuron in the next layer with 10 neurons. But if the next layer only has 10 neurons, it will only pass 10 connections to the next layer.

Something must thus be done about the holes in the data for all smaller layers  as many ML algorithms will not work with NaNs. 

One thought is to impute -1 to the NaNs, and then perform some operation later to "cancel out" these filler calculations. This would involve mathematical techniques that are currently unclear. 

So for now, I have chosen to replace the NaNs with a very improbable small value, understanding that while this may bias the models somewhat, the damage is minimal and the grid is perserved with little added complication. 

This will be revisted in future iterations if better methods are discovered. 

### Column alignment problem for the neuron level
Since a dataframe is "right angled", the largest layer in the network will define an upper boundary for the features. This means only the layer with 20 input neurons will have a full table in the dataframe and all layers in the network will have empty values where the difference between its size and the max layer size would be.

 If we were to index the dataframe to the max_neurons, we would have a sparser table.  

 So it is important to remember that columns 0-19 at the neuron level are  *not a universal point of reference*.
 




## Analyzing Model Metadata 



### Meta-model training 
This next phase involves taking the data from the model and modeling it. This is to help us select which neurons are the most associated with given outputs. We will then use that data to identify our model's proposed "structures". 

The idea is to use machine learning *reflectively* on the results of machine learning, to see if we can find meaningful patterns in the numbers it produces.

#### What is a structure?
The central thesis of this experiment is that we can gain clarity about a model's inner workings by identifing what nodes in the network participate most significantly in contributing to certain outputs. 

If we can identify the "where" (by neural id and layer) and "when" (by epoch, or perhaps in future iterations, batch id) these neurons contribute to certain outputs, we can define a structure associated with that ouput.

This structure would be analogous to identifying what part of the brain is involved in producing language, or recognizing certain shapes, or perceiving sounds, etc. 

These structures are effectively the embodiment of the model's "concept" of the output. 

With the identification of structures, we can then label them, just as we label regions of the brain (Broca's area, the anterior cingulate gyrus, etc). And once we label something, we can begin to reference it, it has a chance to become something we can place programmatic hooks into. We could theoretically delete or change these structures programatically once we identify them. 


### Methods of Analysis 
There is room here for any type of analysis and discretion can be left up to the researcher. The idea is to experiment and find out.

Some options might include:

- Time-series analysis (by observing changes across neural populations or layers across epochs)
- Layer-wise analysis (by observing changes in state of a layer or interactions between layers)
- Aggregate neural analysis (identify pairwise connections, strong correlations between neurons or profile a single neuron's metrics )
- Heatmaps, clustering

The ultimate goal is to apply various machine learning methods experimentally to see if any meaningful patterns can be discovered in network metadata. 

Below will be a sampling of various methods that will periodically be added and expanded. 

In [ ]:

def aggregate_layers_by_type(df):
    # Create a dictionary to store DataFrames for each layer type
    layer_dfs = {}

    # Get all unique layer types
    layer_types = df.index.get_level_values('Layer').unique()
    # Iterate over each layer type
    for layer in layer_types:
        layer_df = df.xs(layer, level='Layer')
        layer_dfs[layer] = layer_df

    return layer_dfs

def aggregate_neurons_across_layers(layer_dfs):
    # Concatenate all DataFrames in layer_dfs
    concatenated_df = pd.concat(layer_dfs.values())

    # Extract neuron types from the index
    concatenated_df.index = concatenated_df.index.map(lambda x: x[1])  # Assuming (Epoch, Neuron) is the index
    
    # Group by neuron type and aggregate
    aggregated_df = concatenated_df.groupby(level=0).mean()  # You can replace mean() with another aggregation method

    return aggregated_df


# def aggregate_neuron_by_type(df):
    # # Ditto neurons 
    # neuron_df = {}
    # print(df)
    # neuron_types = df.index.get_level_values('Neuron').unique()

    # for neuron in neuron_types:
    #     neuron_df = df.xs(neuron, level='neuron')
    # return neuron_df


layer_dfs = aggregate_layers_by_type(model_df)
neuron_dfs = aggregate_neurons_across_layers(layer_dfs)
display(neuron_dfs)

L_input_df = layer_dfs['L_input']
L_hidden_1_df = layer_dfs['L_hidden_1']
L_output_df = layer_dfs['L_output']




# Aggregate by taking the mean across all epochs for the L_input layer
# L_input_avg = L_input_df.groupby(level='Neuron').mean()
# print('avesss', L_input_avg, 'ave ')
# Display the aggregated DataFrame

def compute_epoch_average(df):
    # Group by Layer and Neuron to compute averages across all epochs
    average_df = df.groupby(level=["Layer", "Neuron"]).mean()

    # Create a new index for "Epoch_average" with the same Layer and Neuron combinations
    average_df = average_df.reset_index()
    average_df['Epoch'] = 'Epoch_average'
    average_df.set_index(['Epoch', 'Layer', 'Neuron'], inplace=True)

    # Reset the index of the original dataframe to prepare for merging
    df_reset = df.reset_index()

    # Combine the original data and the average data
    final_df = pd.concat([df_reset, average_df.reset_index()])

    # Set the index back to ["Epoch", "Layer", "Neuron"]
    final_df.set_index(['Epoch', 'Layer', 'Neuron'], inplace=True)

    return final_df



In [4]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

print(model_df.shape)
print(classification_series.shape)

# Now convert the DataFrame to the feature matrix `X` and target `y`
X = model_df.values  # Assuming the features are in the DataFrame
y = classification_series

print(X)

# Step 2: Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Step 3: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the model
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Step 6: Analyze coefficients
coefficients = model.coef_

(3100, 20)
(695,)
[[ 9.79504436e-02  1.63503379e-01 -4.78580929e-02 ... -1.17831320e-01
   6.76362962e-03  3.79307531e-02]
 [ 6.86960295e-02 -6.44231960e-02 -9.53575671e-02 ...  1.14299737e-01
  -5.61272912e-02  1.80215184e-02]
 [ 5.07868789e-02 -1.65503025e-01  1.91396311e-01 ...  1.35015815e-01
  -1.39886677e-01  1.41200006e-01]
 ...
 [ 1.34311736e-01 -3.70444991e-02  1.34768173e-01 ...  1.00000000e-06
   1.00000000e-06  1.00000000e-06]
 [-3.37016433e-01 -7.64986575e-02  2.09974930e-01 ...  1.00000000e-06
   1.00000000e-06  1.00000000e-06]
 [-1.77767456e-01 -3.19150746e-01  1.85033649e-01 ...  1.00000000e-06
   1.00000000e-06  1.00000000e-06]]


ValueError: Found input variables with inconsistent numbers of samples: [3100, 695]